In [1]:
import pandas as pd
import numpy as np


# for Modelling
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense

from tensorflow.keras.models import load_model

# for Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score

In [2]:
df= pd.read_csv('aY_df_all.csv')
df

,Time id,Material,Material_Id,Quantity,Total_Value
0,7,S21-010448-04,41504.0,9.0,3615772.83
1,8,S21-010448-04,41504.0,19.0,7418193.30
2,9,S21-010448-04,41504.0,17.0,6579470.22
3,10,S21-010448-04,41504.0,9.0,3425975.42
4,11,S21-010448-04,41504.0,7.0,2673483.99
...,...,...,...,...,...
3158,35,AC37-000008,86.0,33.0,95237.05
3159,36,AC37-000008,86.0,44.0,156749.66
3160,37,AC37-000008,86.0,44.0,150426.96
3161,38,AC37-000008,86.0,35.0,119426.41


In [3]:
# Define a dictionary to map Time_id to dates
time_id_to_date = {
    1: "2021-01-01", 2: "2021-02-01", 3: "2021-03-01", 4: "2021-04-01", 5: "2021-05-01",
    6: "2021-06-01", 7: "2021-07-01", 8: "2021-08-01", 9: "2021-09-01", 10: "2021-10-01",
    11: "2021-11-01", 12: "2021-12-01", 13: "2022-01-01", 14: "2022-02-01", 15: "2022-03-01",
    16: "2022-04-01", 17: "2022-05-01", 18: "2022-06-01", 19: "2022-07-01", 20: "2022-08-01",
    21: "2022-09-01", 22: "2022-10-01", 23: "2022-11-01", 24: "2022-12-01", 25: "2023-01-01",
    26: "2023-02-01", 27: "2023-03-01", 28: "2023-04-01", 29: "2023-05-01", 30: "2023-06-01",
    31: "2023-07-01", 32: "2023-08-01", 33: "2023-09-01", 34: "2023-10-01", 35: "2023-11-01",
    36: "2023-12-01", 37: "2024-01-01", 38: "2024-02-01", 39: "2024-02-01"
}

# Change Time_id column to dates using the dictionary
df['Time id'] = df['Time id'].map(time_id_to_date)

# Optionally, convert the Time_id column to datetime type
df['Time id'] = pd.to_datetime(df['Time id'])

In [4]:
data = df.copy()

In [6]:
materials=data['Material'].unique()

In [7]:
# Define the evaluate_model function
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mae, r2

# Convert 'Time id' to datetime
data['Time id'] = pd.to_datetime(data['Time id'])

# Define train and test periods
train_start_date = '2021-09-01'
train_end_date = '2023-10-01'
test_start_date = '2023-11-01'
test_end_date = '2024-02-01'

# Define a function to build and train RNN models
def build_and_train_model(model_type, X_train, y_train):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material
for material in data['Material'].unique():
    print(f"Evaluating RNN models for Material: {material}")
    
    # Filter data for the current material
    material_data = data[data['Material'] == material]

    # Split data into train and test sets
    train = material_data[(material_data['Time id'] >= train_start_date) & (material_data['Time id'] <= train_end_date)]
    test = material_data[(material_data['Time id'] >= test_start_date) & (material_data['Time id'] <= test_end_date)]

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train[['Quantity']])
    test_scaled = scaler.transform(test[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    X_test = np.reshape(test_scaled, (test_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity
    y_test = test_scaled[:, 0]

    # Build and train Simple RNN model
    simple_rnn_model = build_and_train_model('SimpleRNN', X_train, y_train)

    # Build and train GRU model
    gru_model = build_and_train_model('GRU', X_train, y_train)

    # Build and train LSTM model
    lstm_model = build_and_train_model('LSTM', X_train, y_train)

    # Evaluate models on test set
    simple_rnn_mae, simple_rnn_r2 = evaluate_model(simple_rnn_model, X_test, y_test)
    gru_mae, gru_r2 = evaluate_model(gru_model, X_test, y_test)
    lstm_mae, lstm_r2 = evaluate_model(lstm_model, X_test, y_test)

    print("Simple RNN MAE:", simple_rnn_mae, " R2 Score:", simple_rnn_r2)
    print("GRU MAE:", gru_mae, " R2 Score:", gru_r2)
    print("LSTM MAE:", lstm_mae, " R2 Score:", lstm_r2)
    print("-" * 50)

Evaluating RNN models for Material: S21-010448-04


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
Simple RNN MAE: 0.11439120753978689  R2 Score: -4.274893072394898
GRU MAE: 0.2796285351117452  R2 Score: -30.58708322379446
LSTM MAE: 0.32877178986867267  R2 Score: -42.642669526432975
--------------------------------------------------
Evaluating RNN models for Material: S21-230126-06


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
Simple RNN MAE: 0.012902386900451465  R2 Score: 0.9529811607802182
GRU MAE: 0.035034779873159214  R2 Score: 0.5443487399229581
LSTM MAE: 0.051749255922105594  R2 Score: 0.03316653316507423
--------------------------------------------------
Evaluating RNN models for Material: S02-060098


...

C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
Simple RNN MAE: 0.15663198133309675  R2 Score: 0.7938513253481757
GRU MAE: 0.21818537016709638  R2 Score: 0.6075797241219183
LSTM MAE: 0.27797008802493406  R2 Score: 0.35825995473804606
--------------------------------------------------
Evaluating RNN models for Material: S01-510164


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 646ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step
Simple RNN MAE: 0.0010644447058439158  R2 Score: 0.9998857096931151
GRU MAE: 0.048150652647018434  R2 Score: 0.8432614124184326
LSTM MAE: 0.07143015563488006  R2 Score: 0.6722278142061647
--------------------------------------------------
Evaluating RNN models for Material: C73-000214


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)

KeyboardInterrupt



In [ ]:
# Convert 'Time id' to datetime
data['Time id'] = pd.to_datetime(data['Time id'])

# Define train period
train_start_date = '2021-09-01'
train_end_date = '2024-02-01'

# Define forecast period
forecast_start_date = '2024-03-01'
forecast_end_date = '2024-06-01'

# Filter data for the train period
train_data = data[(data['Time id'] >= train_start_date) & (data['Time id'] <= train_end_date)]

# Iterate over each material
for material in train_data['Material'].unique():
    print(f"Forecasting for Material: {material}")
    
    # Filter data for the current material
    material_data = train_data[train_data['Material'] == material]

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(material_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Build and train Simple RNN model
    model = Sequential()
    model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

    # Forecast for the next four months
    forecast_dates = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq='MS')
    forecast_values = []

    # Use the trained model to forecast
    for date in forecast_dates:
        # Reshape the date for prediction
        X_forecast = np.array([[date.month / 12]])  # Normalize month to [0, 1]
        X_forecast = np.reshape(X_forecast, (1, 1, 1))

        # Predict using the model
        forecast_value_scaled = model.predict(X_forecast)
        forecast_value = scaler.inverse_transform(forecast_value_scaled)
        forecast_values.append(forecast_value[0, 0])

    # Create DataFrame for the forecasted values
    forecast_df = pd.DataFrame({'Time id': forecast_dates, 'Material': material, 'Forecasted Quantity': forecast_values})
    print(forecast_df)
    print("-" * 50)